## The goal of this simple project is to practice web scraping with BeautifulSoup.  My taret will be Tesla's page on MarketWatch.  I will continue to save this data in an sqlite3 database for future analysis.

### - First let's create a database

In [20]:
import sqlite3 as db
conn = db.connect('tsla.db')
c = conn.cursor()
db.text_factory = str

### - After downloading a csv for the stock data, I will put that in the database and continue to scrape for company data, news headlines, and a few articles for analysis.  Our stock data will include year-to-date statistics.

In [21]:
import pandas as pd
import numpy as np

df = pd.read_csv('tsla.csv')
df

,Date,Open,High,Low,Close,Volume
0,06/16/2023,258.92,263.60,257.21,260.54,"167,915,594"
1,06/15/2023,248.40,258.95,247.29,255.90,"160,171,203"
2,06/14/2023,260.17,261.57,250.50,256.79,"170,575,500"
3,06/13/2023,253.51,259.68,251.34,258.71,"162,384,297"
4,06/12/2023,247.94,250.97,244.59,249.83,"150,740,500"
...,...,...,...,...,...,...
110,01/09/2023,118.96,123.52,117.11,119.77,"190,284,000"
111,01/06/2023,103.00,114.39,101.81,113.06,"220,911,109"
112,01/05/2023,110.51,111.75,107.16,110.34,"157,986,297"
113,01/04/2023,109.11,114.59,107.52,113.64,"180,388,984"


In [22]:
df.to_sql('Tesla 2023', con=conn, if_exists='append', index=False )

115

In [23]:
import requests
import bs4
from bs4 import BeautifulSoup

### - Retriving our html from target url

In [24]:
URL = "https://www.marketwatch.com/investing/stock/tsla?mod=search_symbol"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

### - Now let's zero in on our news column and retrieve most recent article headlines

In [25]:
recentNews = soup.find_all("div", {"class": "article__content"})

### - Now extracting the article headlines and authors from Tesla's overview page.  We will skip articles that do not have authors for analysis purpouses.

In [26]:
titles = []
authors = []
dates = []

for news_element in recentNews:
    
    title_element = news_element.find("a", class_="link")
    author_element = news_element.find("span", class_="article__author")
    provider_element = news_element.find("span", class_="article__provider")
    date_element = news_element.find("span", class_="article__timestamp")
    
    if author_element != None:
        titles.append(title_element.text.strip())
        authors.append(author_element.text.strip())
        dates.append(date_element.text.strip())
    elif provider_element != None:
        titles.append(title_element.text.strip())
        authors.append(provider_element.text.strip())
        dates.append(date_element.text.strip())  

### - We will now put this data into a dataframe in order to store it with our stock data

In [27]:
dict = {'Headline': titles, 'Author': authors, 'Date':dates} 
df2 = pd.DataFrame(dict)
df2['Author'] = df2['Author'].str.slice(start=3)
df2
df2.to_sql('Recent News', con=conn, if_exists='append', index=False )

39

### - Now let's grab the companies general business information, and a list of their top competitors

In [28]:
tslaInfo = soup.find_all("div", {"class": "element element--description description__long"})
info_element = news_element.find("p", class_="description__text")
info_element

In [29]:
tslaComp = soup.find_all("td", {"class": "table__cell w50"})
competitors = []

for competitor in tslaComp:
    
    company = competitor.find("a", class_="link")
    competitors.append(company.text.strip())

In [30]:
competitors

['Toyota Motor Corp.',
 'Volkswagen AG Non-Vtg Pfd.',
 'Volkswagen AG',
 'Mercedes-Benz Group AG',
 'General Motors Co.',
 'Ford Motor Co.',
 'Bayerische Motoren Werke AG',
 'Bayerische Motoren Werke AG Pfd.',
 'NIO Inc. ADR',
 'Stellantis N.V.']

### - Next let's dig a little deeper and grab their board of directors, and some insider transactions

In [31]:
URL2 = "https://www.marketwatch.com/investing/stock/tsla/company-profile?mod=mw_quote_tab"
page2 = requests.get(URL2)
soup2 = BeautifulSoup(page2.content, "html.parser")

bigShots = []

directors = soup2.find_all("li", {"class": "kv__item"})

for director in directors:
    direct_element = director.find("a", class_="primary link")
    role_element = director.find("small", class_="label")
    
    if direct_element != None:
        big = direct_element.text.strip()
        role = role_element.text.strip()
        bigShots.append(big + ' - ' + role)

    
bigShots

['Elon Reeve Musk - Chief Executive Officer & Director',
 'Kimbal Musk - Director',
 'Robyn M Denholm DBA - Chairman',
 'Ira M Ehrenpreis MBA - Independent Director',
 'Joseph Gebbia - Independent Director',
 'Jeffrey B Straubel - Independent Director',
 'James Rupert Murdoch - Independent Director',
 'Kathleen Wilson-Thompson - Independent Director']

### - I'll grab these transaction details and store it in a DataFrame for future analysis

In [33]:
insideMoves = soup2.find_all("td", {"class": "table__cell"})

moves = []

for move in insideMoves:
    move_e = move.find("span", class_="primary")
    typeS = move.find("span", class_="secondary sale")
    typeP = move.find("span", class_="secondary purchase")
    
    if move_e != None:
        moves.append(move_e.text.strip())
    
    if (move_e != None) and (typeS != None):
        s = typeS.text.strip()
        moves.append(s)
        
    elif (move_e != None) and (typeP != None):
        p = typeP.text.strip()
        moves.append(p)
        
        
        
movesF = np.array_split(moves,42)
df3 = pd.DataFrame(movesF, columns = ['Date','Transaction','Name','Type','Shares'])
df3

,Date,Transaction,Name,Type,Shares
0,"Jun 6, 2023",Sale,Zachary J. Kirkhorn,Disposition,"7,403"
1,"Jun 6, 2023",Sale,Vaibhav Taneja,Disposition,"3,181"
2,"Jun 6, 2023",Sale,Xiaotong Zhu,Disposition,794
3,"Jun 6, 2023",Sale,Andrew Baglino,Disposition,"1,299"
4,"Jun 5, 2023",Purchase,Zachary J. Kirkhorn,Derivative/Non-derivative,"13,500"
5,"Jun 5, 2023",Purchase,Zachary J. Kirkhorn,Derivative/Non-derivative,"2,723"
6,"Jun 5, 2023",Purchase,Zachary J. Kirkhorn,Derivative/Non-derivative,648
7,"Jun 5, 2023",Purchase,Vaibhav Taneja,Derivative/Non-derivative,"6,750"
8,"Jun 5, 2023",Purchase,Vaibhav Taneja,Derivative/Non-derivative,388
9,"Jun 5, 2023",Purchase,Xiaotong Zhu,Derivative/Non-derivative,"2,633"


In [34]:
df3.to_sql('Insider Moves', con=conn, if_exists='append', index=False )

42